In [1]:
import pandas as pd
import numpy as np
from numpy import diff
from matplotlib import pyplot as plt
from sklearn.neighbors import KNeighborsClassifier

In [2]:
column_names = ['N', 'X', 'Y']
for i in range(5000):
    column_names.append(str(i))
column_names.append('cluster')
for i in range(1,5):
    column_names.append('c' + str(i))
    
df = pd.read_csv('signals.csv', names=column_names)
df.sort_values(by=['N'], inplace=True)
df.index=df.N
df.drop(['N'], axis=1, inplace=True)
df

,N,X,Y,0,1,2,3,4,5,6,...,4995,4996,4997,4998,4999,cluster,c1,c2,c3,c4
0,168,3642.317,-107.175,0.558,0.595,0.519,0.742,0.664,0.467,0.426,...,0.606,0.523,0.607,0.629,0.664,-1,-1,-1,-1,-1
1,307,4034.298,-668.198,0.650,0.457,0.572,0.644,0.581,0.525,0.645,...,0.397,0.579,0.636,0.570,0.530,-1,-1,-1,-1,-1
2,668,4358.001,1703.199,0.638,0.621,0.646,0.495,0.510,0.611,0.574,...,0.569,0.587,0.541,0.412,0.519,-1,-1,-1,-1,-1
3,229,2786.060,75.125,0.917,0.917,0.916,0.912,0.908,0.905,0.904,...,0.943,0.943,0.943,0.943,0.943,-1,-1,-1,-1,-1
4,316,988.637,-492.056,0.359,0.364,0.379,0.396,0.404,0.405,0.392,...,0.626,0.606,0.629,0.674,0.727,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,498,2648.039,-1611.818,0.633,0.550,0.548,0.531,0.621,0.565,0.628,...,0.515,0.642,0.373,0.517,0.520,-1,-1,-1,-1,-1
756,575,1763.934,67.286,0.842,0.852,0.859,0.865,0.870,0.877,0.876,...,0.990,0.985,0.985,0.986,0.986,-1,-1,-1,-1,-1
757,430,2138.580,1029.394,0.777,0.789,0.800,0.808,0.817,0.824,0.831,...,0.993,0.994,0.995,0.993,0.990,-1,-1,-1,-1,-1
758,722,4078.672,971.900,0.405,0.475,0.503,0.370,0.472,0.481,0.566,...,0.544,0.643,0.489,0.391,0.463,-1,-1,-1,-1,-1


In [3]:
indices = []
dic = {}
for i in df.index:
    differ = diff(np.array(df.loc[i][2:5002]))
    differ_sorted = sorted(differ, key=abs, reverse=True)
    differ_sorted = differ_sorted[:200]

    lis = []
    for n in differ_sorted:
        lis.append(np.where(differ == n)[0])
    dic[i] = lis

In [4]:
def fantastic_four(keyy, thr, flag, numbers=4, second=3000):
    fantastic = []
    con = list(np.concatenate(dic[keyy]))
    if (flag == 'more') or (flag == 'zerotwo'):
        con = [x for x in con if x > thr]
    elif flag == 'less':
        con = [x for x in con if x < thr]
    elif flag == 'edges':
        con = [x for x in con if ((x > thr) and (x < second))]
    elif flag == 'onethree':
        con = [x for x in con if x < second]

    try:
        for j in range(len(con)):
            first = con[0]
            fantastic.append(first)
            exclude = range(first - 70, first + 70)
            for_del = []
            for i in con:
                if i in exclude:
                    for_del.append(i)

            new_list = []
            for n in con:
                if n not in for_del:
                    new_list.append(n)
            con = new_list
    except:
        True

    final = sorted(fantastic[:numbers])
    if flag == 'none':
        return final
    if flag == 'one':
        left = [x for x in fantastic if x < thr]
        right = [x for x in fantastic if x > thr]
        final = [left[0]] + sorted(right[:2])
    if flag == 'two':
        left = [x for x in fantastic if x < thr]
        right = [x for x in fantastic if x > thr]
        final = sorted(left[:2]) + [right[0]]
    if flag == 'middle':
        left = [x for x in fantastic if x < thr]
        right = [x for x in fantastic if x > second]
        final = [left[0]] + [right[0]]
    if flag == 'zerotwo':
        left = [x for x in fantastic if x < second]
        right = [x for x in fantastic if x > second]
        final = [left[0]] + [right[0]]
    if flag == 'onethree':
        left = [x for x in fantastic if x < thr]
        right = [x for x in fantastic if x > thr]
        final = [left[0]] + [right[0]]

    return final

In [5]:
def threshold(list_pos, index):
    if not list_pos:
        keys = list(list_pos.keys())
        return fantastic_four(index, 0, flag='none', numbers=4)
    elif len(list_pos.keys()) == 1:
        keys = list(list_pos.keys())
        if keys[0] == 0:
            return fantastic_four(index, list_pos[keys[0]], flag='more', numbers=3)
        elif keys[0] == 3:
            return fantastic_four(index, list_pos[keys[0]], flag='less', numbers=3)
        elif keys[0] == 1:
            return fantastic_four(index, list_pos[keys[0]], flag='one', numbers=3)
        elif keys[0] == 2:
            return fantastic_four(index, list_pos[keys[0]], flag='two', numbers=3)
    elif len(list_pos.keys()) == 2:
        keys = list(list_pos.keys())
        if (keys[0] == 0) and (keys[1] == 1):
            return fantastic_four(index, list_pos[keys[1]], flag='more', numbers=2, second=list_pos[keys[0]])
        if (keys[0] == 2) and (keys[1] == 3):
            return fantastic_four(index, list_pos[keys[0]], flag='less', numbers=2, second=list_pos[keys[1]])
        if (keys[0] == 1) and (keys[1] == 2):
            return fantastic_four(index, list_pos[keys[0]], flag='middle', numbers=2, second=list_pos[keys[1]])
        if (keys[0] == 0) and (keys[1] == 3):
            return fantastic_four(index, list_pos[keys[0]], flag='edges', numbers=2, second=list_pos[keys[1]])
        if (keys[0] == 0) and (keys[1] == 2):
            return fantastic_four(index, list_pos[keys[0]], flag='zerotwo', numbers=2, second=list_pos[keys[1]])
        if (keys[0] == 1) and (keys[1] == 3):
            return fantastic_four(index, list_pos[keys[0]], flag='onethree', numbers=2, second=list_pos[keys[1]])

In [6]:
exp = df[['c1', 'c2', 'c3', 'c4']]
for index, row in exp.iterrows():
    old_values, all_values = [], []
    positions = {}
    for pos, val in enumerate(row):
        if val != -1:
            old_values.append(val)
            positions[pos] = val
    new_values = threshold(positions, index)

    all_values = [*old_values, *new_values]
    all_values.sort()

    df.loc[index, ['c1', 'c2', 'c3', 'c4']] = all_values

In [84]:
labeled = df[df.cluster != -1]

X, y = labeled.iloc[:,1:3].to_numpy(), labeled.cluster.to_numpy()

clf = KNeighborsClassifier(9)
clf.fit(X, y)
cluster_predictions = clf.predict(df.iloc[:,1:3].to_numpy())

for i in range(df.shape[0]):
    if df.cluster.iloc[i] == -1:
        df.cluster.iloc[i] = cluster_predictions[i]

In [ ]:
df.to_csv('result.csv', index=False, header=None)

In [7]:
df

,N,X,Y,0,1,2,3,4,5,6,...,4995,4996,4997,4998,4999,cluster,c1,c2,c3,c4
0,168,3642.317,-107.175,0.558,0.595,0.519,0.742,0.664,0.467,0.426,...,0.606,0.523,0.607,0.629,0.664,-1,0,1019,2840,3174
1,307,4034.298,-668.198,0.650,0.457,0.572,0.644,0.581,0.525,0.645,...,0.397,0.579,0.636,0.570,0.530,-1,0,218,3529,4809
2,668,4358.001,1703.199,0.638,0.621,0.646,0.495,0.510,0.611,0.574,...,0.569,0.587,0.541,0.412,0.519,-1,0,640,1574,3669
3,229,2786.060,75.125,0.917,0.917,0.916,0.912,0.908,0.905,0.904,...,0.943,0.943,0.943,0.943,0.943,-1,0,3578,3805,4084
4,316,988.637,-492.056,0.359,0.364,0.379,0.396,0.404,0.405,0.392,...,0.626,0.606,0.629,0.674,0.727,-1,0,1827,1938,2042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,498,2648.039,-1611.818,0.633,0.550,0.548,0.531,0.621,0.565,0.628,...,0.515,0.642,0.373,0.517,0.520,-1,0,1994,3433,4882
756,575,1763.934,67.286,0.842,0.852,0.859,0.865,0.870,0.877,0.876,...,0.990,0.985,0.985,0.986,0.986,-1,0,231,4287,4390
757,430,2138.580,1029.394,0.777,0.789,0.800,0.808,0.817,0.824,0.831,...,0.993,0.994,0.995,0.993,0.990,-1,0,4225,4434,4584
758,722,4078.672,971.900,0.405,0.475,0.503,0.370,0.472,0.481,0.566,...,0.544,0.643,0.489,0.391,0.463,-1,0,371,3460,3784
